In [ ]:
pip install langchain unstructured openai chromadb

In [24]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader, UnstructuredFileLoader
from langchain.prompts.chat import HumanMessagePromptTemplate
import os
# os.environ['OPENAI_API_KEY'] = '...'
chat=ChatOpenAI(temperature=0)

In [2]:
# loader = DirectoryLoader('./data/', glob='**/*.txt')
loader = UnstructuredFileLoader('./data/labor_standard_act.txt')
docs = loader.load()
print(len(docs))

1


In [12]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
docsearch = Chroma.from_documents(texts, embeddings, persist_directory="./cache")

Running Chroma using direct local API.
loaded in 0 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection


In [13]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch)
# from langchain.chains import RetrievalQAWithSourcesChain
# qa = RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="map_reduce", retriever=docsearch.as_retriever())

In [26]:
human_message_prompt=HumanMessagePromptTemplate(prompt=PromptTemplate(
    template="Translate the following input to English: {text}",
    input_variables=["text"],
))
human_message=human_message_prompt.format_messages(text="裁員時不通知員工是否合法？")
q=chat(human_message)
print(q.content)

Is it legal to not inform employees when downsizing?


In [29]:
a=qa.run(q.content)
print(a)
# qa({"question": "if I worked for two years, how long should I notify the company before I resign?"}, return_only_outputs=True)

 No, it is not legal to not inform employees when downsizing. According to Article 11 of the labor laws, employers must provide advance notice to workers when downsizing.


In [31]:
human_message_prompt=HumanMessagePromptTemplate(prompt=PromptTemplate(
    template="Translate the following input to Traditional Chinese: {text}",
    input_variables=["text"],
))
human_message=human_message_prompt.format_messages(text=a)
output=chat(human_message)
print(output.content)

不，不通知員工裁員是不合法的。根據勞動法第11條，雇主在裁員時必須提前通知工人。
